In [ ]:
try:
    from google.colab import drive, files
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    import sys
    drive.mount('/content/gdrive', force_remount=True)
    sys.path.append("./gdrive/My Drive/Colab Notebooks/")

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import scipy.io as io
import matplotlib.pyplot as plt
import os
import time
import datetime
import copy
import dataclasses
from dataclasses import dataclass, field
import itertools
import json

from smfret.tf_layers import Attention
from smfret.tf_layers import Conv
from smfret.tf_layers import Summary
from smfret.tf_layers import PrependTaskToken
from smfret.tf_layers import Reconstructor
from smfret.tf_layers import Embedding
from smfret.tf_layers import PositionEmbedding

from smfret.trace_simulator import Simulator
from smfret.trace_simulator import ParameterGenerator
from smfret.trace_simulator import SimulatedTraceSet

from smfret.multi_task_learning import FRETStateTraceSet
from smfret.multi_task_learning import SavedTraceSet

from smfret.dataset import MatlabTraceSet
from smfret.dataset import FRETTraceSet
from smfret.learning_task import SimpleTask

from smfret.tf_learning import LRSchedule

from tqdm.auto import tqdm
from IPython import display

In [ ]:
@dataclass(eq=True, frozen=True)
class NetworkParams:
    """Class for storing hyper parameters."""
    conv_width: int
    inner_dim: int
    dense_width_array: int
    dense_activation: int
    attention_num_heads: int
    attention_key_dim: int
    attention_depth: int
    max_lr: float
    warmup_steps: int
    max_iteration: int
    batch_size: int
    evolve_rate_per_iteration: int
    use_layer_norm: bool = False
    use_checkpoint: bool = False
    checkpoint_path: str = ''
    save_model_path: str = field(default='', compare = False)
    tasks: tuple = ()
    grad_clip_norm: float = 1e-2
    dtype: str = 'bfloat16'
    dropout: float = 0.0


def build_conv(params):
    """Builds the conv layer of the NN."""
    conv = Conv(width=params.conv_width, inner_dim=params.inner_dim)
    return conv


def build_transformer(params):
    """Builds the transformer components of the NN."""
    base_seq_input = keras.Input(shape=(None, params.inner_dim))
    positional_embedding = PositionEmbedding()
    seq_input = tf.cast(base_seq_input, params.dtype) + positional_embedding(base_seq_input)
    for _ in range(params.attention_depth):
        seq_input_reidule = seq_input

        if params.use_layer_norm:
            ln = tf.keras.layers.LayerNormalization()
            seq_input = ln(seq_input)
        attention = keras.layers.MultiHeadAttention(num_heads=params.attention_num_heads, key_dim=params.inner_dim)
        dropout = keras.layers.Dropout(params.dropout)
        seq_input = dropout(attention(seq_input, seq_input))

        seq_input += seq_input_reidule
        seq_input_reidule = seq_input

        if params.use_layer_norm:
            ln = tf.keras.layers.LayerNormalization()
            seq_input = ln(seq_input)

        for dense_unit in [*params.dense_width_array, params.inner_dim]:
            dense_ff = keras.layers.Dense(units=dense_unit, activation=params.dense_activation)
            dropout = keras.layers.Dropout(params.dropout)
            seq_input = dropout(dense_ff(seq_input))

        seq_input += seq_input_reidule

        if not params.use_layer_norm:
            seq_input = tf.math.l2_normalize(seq_input, axis=-1)

    seq_output = seq_input
    transformer = tf.keras.Model(inputs=base_seq_input, outputs=seq_output)
    return transformer

def build_reconstructor(params):
    """Builds the reconstructor that maps a stride window back to each frame."""
    reconstructor = Reconstructor(width=params.conv_width)
    return reconstructor

In [ ]:
refresh = True

if refresh:
    models = {}

types = ['NBD', 'NBDQUICK', 'TRANSITIONPROBQUICK', 'FRETSTATEHIGHEST', 'FRETSTATELOWEST','TRANSITIONRATE', 'DWELLTIME', 'FRETSTATECOUNT', 'NOISELEVEL',  'MULTISTEP', 'DONORLIFETIME', 'ACCEPTORLIFETIME',
         'SELECTEDSEGMENTS', 'REEMERGENT', 'REEMERGENTBINARY', 'FRETSTATE', 'HIGHFRETBINARY', 'LOWFRETBINARY', 'QUICKTRANSITION', 'LOWFRETSELECTED', 'BACKGROUND', 'COINCIDENCE', 'SINGLECHANNELSTATE',
         'SINGLECHANNELSTATEFASTTRANSITION', 'SINGLECHANNELDWELLTIME', 'SINGLECHANNELBLINKING', 'SINGLECHANNELLIFETIME', 'SINGLECHANNELNOISELEVEL', 'SINGLECHANNELPHOTOBLEACHINGSTEPSFRAME',
         'SINGLECHANNELPHOTOBLEACHINGSTEPSTRACE',]
framewise_types = ['FRETSTATE', 'SELECTEDSEGMENTS', 'REEMERGENT', 'REEMERGENTBINARY', 'HIGHFRETBINARY', 'LOWFRETBINARY', 'QUICKTRANSITION', 'LOWFRETSELECTED',
                   'SINGLECHANNELSTATE', 'SINGLECHANNELSTATEFASTTRANSITION', 'SINGLECHANNELBLINKING', 'SINGLECHANNELPHOTOBLEACHINGSTEPSFRAME']

n_categories = {
    'FRETSTATE': 12,
    'FRETSTATECOUNT': 5,
    'FRETSTATEHIGHEST': 12,
    'FRETSTATELOWEST': 12,
    'DWELLTIME': 12,
    'NOISELEVEL': 6,
    'DONORLIFETIME': 6,
    'ACCEPTORLIFETIME': 6,
    'MULTISTEP': 4,
    'SELECTEDSEGMENTS': 2,
    'TRANSITIONRATE': 6,
    'REEMERGENT': 6,
    'REEMERGENTBINARY': 2,
    'HIGHFRETBINARY': 2,
    'LOWFRETBINARY': 2,
    'NBD': 10,
    'QUICKTRANSITION': 2,
    'NBDQUICK': 10,
    'TRANSITIONPROBQUICK': 6,
    'LOWFRETSELECTED': 2,
    'BACKGROUND': 12,
    'COINCIDENCE': 2,
    'SINGLECHANNELSTATE': 12,
    'SINGLECHANNELSTATEFASTTRANSITION': 12,
    'SINGLECHANNELDWELLTIME': 22,
    'SINGLECHANNELBLINKING': 2,
    'SINGLECHANNELLIFETIME': 12,
    'SINGLECHANNELNOISELEVEL': 10,
    'SINGLECHANNELPHOTOBLEACHINGSTEPSFRAME': 8,
    'SINGLECHANNELPHOTOBLEACHINGSTEPSTRACE': 8,
}

TYPES = types
FRAMEWISE_TYPES = framewise_types
N_CATEGORIES = n_categories

def prepare_models(params):
    """Prepares all models for every task."""

    if IN_COLAB and params.dtype == 'float16':
      keras.mixed_precision.set_global_policy('mixed_float16')
    elif IN_COLAB and params.dtype == 'bfloat16':
      keras.mixed_precision.set_global_policy('mixed_bfloat16')
    else:
      keras.mixed_precision.set_global_policy('float32')

    general_token = PrependTaskToken(params.inner_dim)
    task_token_assignment = {name: general_token for name in types}

    if params.use_checkpoint:
      saved_model = keras.models.load_model(f'{params.checkpoint_path}/{params.tasks[0]}.h5', compile=False, custom_objects={'PositionEmbedding': PositionEmbedding})
      saved_embedding = saved_model.layers[-2]
      conv = saved_embedding.conv
      transformer = saved_embedding.transformer
      reconstructor = saved_embedding.reconstructor
    else:
      conv = build_conv(params)
      transformer = build_transformer(params)
      reconstructor = build_reconstructor(params)


    models = {}

    for type_name in params.tasks:

        if params.use_checkpoint:
          saved_model = keras.models.load_model(f'{params.checkpoint_path}/{type_name}.h5', compile=False, custom_objects={'PositionEmbedding': PositionEmbedding})
          saved_embedding = saved_model.layers[-2]
          prepend_task_token = saved_embedding.prepend_task_token
          readout = saved_model.layers[-1]

        else:
          readout = keras.layers.Dense(units=N_CATEGORIES[type_name])
          prepend_task_token = task_token_assignment[type_name]

        embedding = Embedding(
            conv=conv,
            transformer=transformer,
            prepend_task_token=prepend_task_token,
            reconstructor=reconstructor,
            framewise=(type_name in FRAMEWISE_TYPES),
        )

        base_input = keras.Input(shape=(None, 4))
        embedding_vector = embedding(base_input)
        logits = readout(embedding_vector)

        model = keras.Model(inputs=base_input, outputs=logits)
        model.compile(
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        )

        if type_name not in models:
            models[type_name] = model

    return models

In [ ]:
keras.losses.SparseCategoricalCrossentropy(from_logits=True)(tf.constant([1.0], dtype=tf.float32), tf.constant([[0., 0.]], dtype=tf.bfloat16), )

<tf.Tensor: shape=(), dtype=bfloat16, numpy=0.691406>

In [ ]:
def read_data(params):
    """Reads training and val datasets."""
    if IN_COLAB:
      path = './gdrive/My Drive/Data/'
    else:
      path = 'saved_dataset/'
    train_sets = {}

    for name, _ in tqdm(N_CATEGORIES.items(), desc='Training Dataset'):
        file = path + f'train/{name}.npz'
        train_sets[name] = SavedTraceSet(size=params.batch_size, file=file)

    val_sets = {}
    val_size = 1000
    for name, _ in tqdm(n_categories.items(), desc='Val Dataset'):
        file = path + f'eval/{name}.npz'
        val_sets[name] = SavedTraceSet(size=val_size, file=file)

    return train_sets, val_sets

In [ ]:
VAL_FREQUENCY = 100
PLOT_CLEAR_FREQUENCY = 100
VARS = None
@dataclass(eq=True, frozen=True)
class TrainResults:
    """Stores results from model training."""
    val_loss_history: np.ndarray
    train_loss_history: np.ndarray
    val_task_loss_history: dict
    train_task_loss_history: dict

    def min_val_loss(self):
        """Calculates minimum validation loss."""
        return np.min(val_loss_history)


def combine_grad(model_vars_list, grads_list):
    """Combines mulitple calculations of grad for a single update step."""
    combined_model_vars = []
    combined_grads = []
    positions = {}

    for model_vars, grads in zip(model_vars_list, grads_list):
        for var, grad in zip(model_vars, grads):
            ref = var.ref()
            if ref in positions:
                pos = positions[ref]
                if combined_grads[pos] is None:
                    combined_grads[pos] = grad
                elif grad is not None:
                    combined_grads[pos] += grad
            else:
                combined_grads.append(grad)
                combined_model_vars.append(var)
                positions[ref] = len(combined_model_vars) - 1

    return (combined_model_vars, combined_grads)


def clip_grad_by_norm(grad, clip_norm):
    """Clips the gradient by its norm."""
    if grad is None:
        return None
    else:
        return tf.clip_by_norm(grad, clip_norm)


def train(params, models, train_sets, val_sets, save_models=True):
    """Iterates through the data and trains the models."""
    plt.figure(dpi=120)

    lr = LRSchedule(
        max_learning_rate=params.max_lr,
        warmup_steps=params.warmup_steps,
        decay_steps=int(params.max_iteration * 1.1),
    )

    val_loss_history = []
    train_loss_history = []
    val_task_loss_history = {}
    train_task_loss_history = {}

    optimizer = tf.keras.optimizers.Adam(lr, jit_compile=True)

    if params.dtype == 'float16':
        optimizer = keras.mixed_precision.LossScaleOptimizer(optimizer, dynamic=False, initial_scale=1024.0)
        scale_loss = optimizer.get_scaled_loss
        unscale_grads = optimizer.get_unscaled_gradients
    else:
        scale_loss = lambda x: x
        unscale_grads = lambda x: x

    def train_step(dataset, model, task_name, training):
        """Runs ne step of training and returns loss."""
        task = SimpleTask(dataset.to_tensor(), dataset.label, model, task_name)
        return train_step_core(task.model, task.feature_tensor, task.labels, training)

    @tf.function
    def train_step_core(model, feature_tensor, labels, training):
        """Core of the training step."""
        y_pred = model(feature_tensor, training=training)
        y_true = tf.cast(labels, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        loss = model.compiled_loss(y_true, y_pred)
        return loss

    for i in tqdm(range(params.max_iteration)):
        total_loss = 0.0
        model_vars_list = []
        grads_list = []
        for task_name in params.tasks:
            with tf.GradientTape() as tape:
                loss = train_step(train_sets[task_name], models[task_name], task_name, training=True)
                float16_safe_loss = scale_loss(loss)

            if task_name not in train_task_loss_history:
                train_task_loss_history[task_name] = [loss.numpy()]
            else:
                train_task_loss_history[task_name].append(loss.numpy())

            total_loss += loss
            train_sets[task_name].evolve(params.evolve_rate_per_iteration)
            model_vars = tape.watched_variables()
            grads = unscale_grads(tape.gradient(float16_safe_loss, model_vars))
            grads = [clip_grad_by_norm(g, params.grad_clip_norm) for g in grads]
            model_vars_list.append(model_vars)
            grads_list.append(grads)

        model_vars, grads = combine_grad(model_vars_list, grads_list)
        train_loss_history.append(total_loss.numpy())

        optimizer.apply_gradients(zip(grads, model_vars))

        # if i % PLOT_CLEAR_FREQUENCY == 0:
        #   plt.clf()
        #   plt.figure(dpi=120)

        # plt.subplot(1, 1, 1)
        # color = plt.cm.tab20(0)
        # plt.scatter(i, total_loss.numpy(), color=color, marker='.')
        # ax_right = plt.gca().twinx()
        # plt.title(f'Iter {i + 1} out of {params.max_iteration}')
        # plt.xlabel('Iteration')
        # plt.gca().set_ylabel('Total Loss')
        # plt.grid()
        # if i % PLOT_CLEAR_FREQUENCY == 0:
        #     for color_idx, task_name in enumerate(train_task_loss_history):
        #         loss = train_task_loss_history[task_name]
        #         color = plt.cm.tab20(color_idx + 1)
        #         ax_right.plot(loss, color=color, linewidth=0.5, label=task_name)
        # ax_right.set_ylabel('Task Loss')
        # ax_right.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
        #   fancybox=True, shadow=True, ncol=2)

        if i % VAL_FREQUENCY == 0:
            val_loss = 0.0
            for task_name in params.tasks:
                val_loss += train_step(val_sets[task_name], models[task_name], task_name, training=False)
                if task_name not in val_task_loss_history:
                    val_task_loss_history[task_name] = [val_loss.numpy()]
                else:
                    val_task_loss_history[task_name].append(val_loss.numpy())
            print('val loss =', val_loss.numpy())
            val_loss_history.append(val_loss.numpy())

            # plt.subplot(1, 1, 1)
            # color = plt.cm.tab20(2)
            # plt.scatter(i, val_loss.numpy(), color=color, marker='.')

        if save_models:
            for task_name in params.tasks:
                if i % VAL_FREQUENCY == 0 and i > 0:
                    if np.min(val_loss_history) == val_loss.numpy():
                        models[task_name].save(f'{params.save_model_path.rstrip("/")}/{task_name}.h5')

        # display.clear_output(wait=True)
        # display.display(plt.gcf())

    train_results = TrainResults(
        train_loss_history=train_loss_history,
        val_loss_history=val_loss_history,
        val_task_loss_history=val_task_loss_history,
        train_task_loss_history=train_task_loss_history,
    )

    return train_results

In [ ]:
params = NetworkParams(
    conv_width=100,
    inner_dim=32,
    dense_width_array=(64, 32, 16),
    dense_activation='relu',
    attention_num_heads=8,
    attention_key_dim=16,
    attention_depth=5,
    max_lr=1e-3,
    warmup_steps=50,
    max_iteration=100,
    batch_size=128,
    evolve_rate_per_iteration=32,
    tasks=['FRETSTATE'],
)

with tf.device('/CPU:0'):
  train_sets, val_sets = read_data(params)

Training Dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Val Dataset:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
def load_log(file):
    """Loads the log of a training experiment."""
    with open(file) as f:
        r = json.load(f)
    return r

In [ ]:
lr = [1e-4]
attention_depth = [4]
dense_activation = ['gelu']
inner_dim = [96]
batch_size = [4096]
conv_width = [100]
dropout = [0.1,]

tasks = [', '.join(types)]

tasks = [tuple(task.replace(' ', '').split(',')) for task in tasks]

options = itertools.product(batch_size, inner_dim, dense_activation, attention_depth, lr, dropout, conv_width, tasks)

In [ ]:
USE_EXISTING_LOG = False

if USE_EXISTING_LOG:
    log_path = './gdrive/My Drive/Data/logs/hptuning_20231202-172959.json'
    hp_tuning_results = load_log(log_path)
else:
    hp_tuning_results = []

timestr = time.strftime("%Y%m%d-%H%M%S")
if IN_COLAB:
    if not USE_EXISTING_LOG:
        log_path = f'./gdrive/My Drive/Data/logs/hptuning_{timestr}.json'
        print(log_path)
    saved_model_path_format = '/content/gdrive/MyDrive/Data/saved_models/model-{}/'
else:
    log_path = f'logs/hptuning_{timestr}.json'
    saved_model_path_format = 'saved_models/model-{}/'

with tf.device('/GPU:0'):
    for option in options:
        batch_size, inner_dim, dense_activation, attention_depth, lr, dropout, conv_width, tasks = option
        saved_model_path = saved_model_path_format.format(time.strftime('%Y%m%d-%H%M%S'))
        params = NetworkParams(
            conv_width=conv_width,
            inner_dim=inner_dim,
            dense_width_array=[4 * inner_dim, 4 * inner_dim],
            dense_activation=dense_activation,
            attention_num_heads=4,
            attention_key_dim=inner_dim // 4,
            attention_depth=attention_depth,
            max_lr=lr,
            warmup_steps=100,
            max_iteration=50002,
            batch_size=batch_size,
            evolve_rate_per_iteration=batch_size // 2,
            use_layer_norm=True,
            use_checkpoint=True,
            checkpoint_path='/content/gdrive/MyDrive/Data/saved_models/model-20240110-155852',
            save_model_path=saved_model_path,
            tasks=tasks,
            dtype='bfloat16',
            dropout=dropout,
        )

        existing_params = [NetworkParams(**eval(e)[0]) for e in hp_tuning_results]

        if params in existing_params:
            print('Existing condition detected, skipped re-training!')
            continue

        for name in train_sets:
            train_sets[name].resize(params.batch_size)

        models = prepare_models(params)
        train_results = train(params, models, train_sets, val_sets, save_models=True)
        hp_tuning_results.append(
            str((dataclasses.asdict(params), dataclasses.asdict(train_results)))
        )
        with open(log_path, 'w') as f:
            json.dump(hp_tuning_results, f)

        del models